In [51]:
from rdkit import Chem
import pandas as pd

# Read the CSV file containing SMILES strings
df_smiles = pd.read_csv('smiles.csv')

# Function to convert SMILES string to RDKit molecule
def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol

# Add the 'ROMol' column by applying the smiles_to_mol function
df_smiles['ROMol'] = df_smiles['smiles'].apply(smiles_to_mol)

# Add the 'n_atoms' column to store the number of atoms in each molecule
df_smiles['n_atoms'] = df_smiles['ROMol'].apply(lambda mol: mol.GetNumAtoms() if mol else None)

# Display the DataFrame
df_smiles[['type', 'smiles', 'n_atoms']]


,type,smiles,n_atoms
0,Ethene,C=C,2
1,benzene,C1=CC=CC=C1,6
2,Ethanol,CCO,3
3,formaldehyde,C=O,2
4,carbone dioxide,O=C=O,3


In [52]:
import os
import sys
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
sys.path.append('/Users/iwatobipen/develop/chemoenv/psikit/psikit/')

In [53]:
from psikit import Psikit

In [54]:
pk = Psikit(debug=True,threads=1,memory=12)

In [57]:
# 1. Import Required Libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import psi4
# 2. Read the SMILES data from your existing 'smiles.csv' file
smiles_df = pd.read_csv("smiles.csv")
# Check the first few rows of the dataframe to ensure it's loaded correctly
print(smiles_df.head())
# 3. Convert SMILES to RDKit Molecules and Compute Atom Counts
def add_chemical_info(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None
    n_atoms = mol.GetNumAtoms()
    return mol, n_atoms
smiles_df['ROMol'], smiles_df['n_atoms'] = zip(*smiles_df['smiles'].apply(add_chemical_info))
# 4. Generate 3D Coordinates for RDKit Molecules (If Not Already Present)
def generate_3d_coordinates(mol):
    if mol is None:  # Skip invalid molecules
        return None
    try:
        mol = Chem.AddHs(mol)  # Add hydrogens first (required for 3D generation)
        success = AllChem.EmbedMolecule(mol, randomSeed=42)  # Generate 3D coordinates
        if success != 0:
            print(f"Warning: 3D generation failed for {Chem.MolToSmiles(mol)}")
            return None
        AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry using UFF (Universal Force Field)
        return mol
    except Exception as e:
        print(f"Error generating 3D coordinates for molecule: {e}")
        return None
# Apply 3D coordinate generation to all molecules
smiles_df['3D_mol'] = smiles_df['ROMol'].apply(generate_3d_coordinates)
# 5. Check the DataFrame with the newly added 'ROMol', 'n_atoms', and '3D_mol' columns
print(smiles_df.head())
# 6. Convert RDKit Molecule to XYZ Format
def mol_to_xyz(mol):
    if mol is None:
        return None
    try:
        mol_xyz = Chem.MolToXYZBlock(mol)  # Convert to XYZ format
        return mol_xyz
    except Exception as e:
        print(f"Error converting RDKit molecule to XYZ: {e}")
        return None
# Convert molecules to XYZ format
smiles_df['xyz'] = smiles_df['3D_mol'].apply(mol_to_xyz)
# 7. Set Up Psi4 for Energy Optimization
def compute_energy(mol_xyz):
    if mol_xyz is None:
        return None
    try:
        # Use the XYZ format with Psi4
        psi4.geometry(mol_xyz)  # Define the molecule for Psi4
        energy = psi4.energy('scf/6-31G(d)')  # Perform energy calculation (single-point)
        return energy
    except Exception as e:
        print(f"Error calculating energy: {e}")
        return None
# 8. Perform Energy Calculations and Add to DataFrame
# We will now add the energy values as a new column to the DataFrame.
smiles_df['energy'] = smiles_df['xyz'].apply(compute_energy)

              type       smiles
0           Ethene          C=C
1          benzene  C1=CC=CC=C1
2          Ethanol          CCO
3     formaldehyde          C=O
4  carbone dioxide        O=C=O
              type       smiles  \
0           Ethene          C=C   
1          benzene  C1=CC=CC=C1   
2          Ethanol          CCO   
3     formaldehyde          C=O   
4  carbone dioxide        O=C=O   

                                              ROMol  n_atoms  \
0  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5970>        2   
1  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5820>        6   
2  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5d60>        3   
3  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a56d0>        2   
4  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5740>        3   

                                             3D_mol  
0  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5dd0>  
1  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a5f20>  
2  <rdkit.Chem.rdchem.Mol object at 0x14f84e2a57b0

In [58]:
# 9. Display the DataFrame with Molecules and Their Energies
print(smiles_df[['smiles', 'n_atoms', 'energy']])

        smiles  n_atoms      energy
0          C=C        2  -78.030622
1  C1=CC=CC=C1        6 -230.701592
2          CCO        3 -154.070191
3          C=O        2 -113.863627
4        O=C=O        3 -187.597977
